# Test of native TF/Keras implementation

In [1]:
import random
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, SpatialDropout1D, BatchNormalization, Dropout
from tensorflow.keras.metrics import AUC

2021-11-28 02:47:18.778864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 02:47:18.778951: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Global Configs

In [2]:
RANDOM_SEED = 1919
BATCH_SIZE = 128
SEQUENCE_SIZE = 10
EMBEDDING_SIZE = 30
NUMBER_SAMPLES = 10000

## Setup Synthetic Data

In [3]:
# setup for reproducible data
np.random.seed(13)

#create artificial training data
X = np.random.normal(size = (NUMBER_SAMPLES, SEQUENCE_SIZE, EMBEDDING_SIZE)).astype(np.float32)
y = np.random.binomial(1, 0.3, size = NUMBER_SAMPLES).astype(np.float32)

In [4]:
print(f'X shape {X.shape}, y shape {y.shape}')

X shape (10000, 10, 30), y shape (10000,)


## Reproducibility Helper function

In [5]:
# setup for reproducible training
def set_for_reproducibility():
    # ensure clean setup of TF
    tf.keras.backend.clear_session()

    # set various RNG seed
    np.random.seed(RANDOM_SEED)
    tf.random.set_seed(RANDOM_SEED)
    random.seed(RANDOM_SEED)

    # single-thread execution for reproducibility at the expense of run-time
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

##  test original 3-d Structure

In [6]:
# setup for reproducible training
set_for_reproducibility()

In [7]:
# In[ ]:


#build and compile TF model


def nn():
    model_input = Input(shape = (SEQUENCE_SIZE, EMBEDDING_SIZE))
    model_dropout1 = SpatialDropout1D(0.2)(model_input)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [8]:
# In[ ]:

model = nn()

print('\nBaseline model training')
model.fit(X, y, epochs=5, batch_size=BATCH_SIZE, shuffle=False)

2021-11-28 02:47:21.801155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-28 02:47:21.801208: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-28 02:47:21.801244: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dask-notebook-0): /proc/driver/nvidia/version does not exist
2021-11-28 02:47:21.801502: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



Baseline model training
Epoch 1/5
79/79 [==============================] - 7s 42ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 3s 44ms/step - loss: 0.6393 - auc: 0.5234
Epoch 4/5
79/79 [==============================] - 3s 44ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 48ms/step - loss: 0.6259 - auc: 0.5411


## Test conversion to 2-d structure

In [9]:
# setup for reproducible training
set_for_reproducibility()

In [10]:
# setup model for 2-d structure


def nn2():
    model_input = Input(shape = (SEQUENCE_SIZE * EMBEDDING_SIZE,))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, SEQUENCE_SIZE, EMBEDDING_SIZE))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [11]:
model2 = nn2()

X_2d = X.reshape(-1, SEQUENCE_SIZE * EMBEDDING_SIZE)
print('\n2-d model training')
model2.fit(X_2d, y, epochs=5, batch_size=BATCH_SIZE, shuffle=False)


2-d model training
Epoch 1/5
79/79 [==============================] - 7s 49ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 4s 52ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6392 - auc: 0.5235
Epoch 4/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 47ms/step - loss: 0.6259 - auc: 0.5411: 0s - loss: 0.6258 - auc


## Compared learned parameters from the two models

In [12]:
## Compare learned parameters
print('compare baseline model vs 2-d model')
for nn_parameter, nn2_parameter in zip(model.weights, model2.weights):
    print(
        f'{np.allclose(nn_parameter.numpy(), nn2_parameter.numpy())}, '
        f'max diff {np.max(np.abs(nn_parameter.numpy() - nn2_parameter.numpy()))}'
    )

compare baseline model vs 2-d model
False, max diff 0.0006649121642112732
False, max diff 0.0010492801666259766
False, max diff 0.0005425214767456055
False, max diff 0.00015504658222198486
False, max diff 0.0003531612455844879
False, max diff 9.554624557495117e-05
False, max diff 6.882846355438232e-05
False, max diff 0.000373058021068573
False, max diff 3.874674439430237e-05
False, max diff 0.00011435151100158691
False, max diff 6.034970283508301e-06


## Test 2-d structure with scikeras.KerasClassifer

In [13]:
from scikeras.wrappers import KerasClassifier

In [14]:
# setup for reproducible training
set_for_reproducibility()

In [15]:
# setup model for 2-d structure


def nn_scikeras():
    model_input = Input(shape = (SEQUENCE_SIZE * EMBEDDING_SIZE,))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, SEQUENCE_SIZE, EMBEDDING_SIZE))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [16]:
model_scikeras = KerasClassifier(model=nn_scikeras, optimizer='Adam')

X_2d = X.reshape(-1, SEQUENCE_SIZE * EMBEDDING_SIZE)
print('\nscikeras model training')
model_scikeras.fit(X_2d, y, epochs=5, batch_size=BATCH_SIZE, shuffle=False)


scikeras model training
Epoch 1/5
79/79 [==============================] - 7s 51ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 4s 49ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 4s 48ms/step - loss: 0.6392 - auc: 0.5235
Epoch 4/5
79/79 [==============================] - 4s 47ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6259 - auc: 0.5411


KerasClassifier(
	model=<function nn_scikeras at 0x7fd89e65a5e0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=Adam
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

## Compared learned parameters from the two models

In [17]:
## Compare learned parameters
print('compare baseline model vs scikeras model')
for nn_parameter, scikeras_parameter in zip(model.weights, model_scikeras.model_.weights):
       print(
        f'{np.allclose(nn_parameter.numpy(), scikeras_parameter.numpy())}, '
        f'max diff {np.max(np.abs(nn_parameter.numpy() - scikeras_parameter.numpy()))}'
    )

compare baseline model vs scikeras model
False, max diff 0.0006649121642112732
False, max diff 0.0010492801666259766
False, max diff 0.0005425214767456055
False, max diff 0.00015504658222198486
False, max diff 0.0003531612455844879
False, max diff 9.554624557495117e-05
False, max diff 6.882846355438232e-05
False, max diff 0.000373058021068573
False, max diff 3.874674439430237e-05
False, max diff 0.00011435151100158691
False, max diff 6.034970283508301e-06
